# Лабораторна робота №1. Дослідження директорної антени.
Лесняк Андрій, 569Мн;  
5 варіант.  

Частота f = 480 МГц  
Число елементів антени N = 12

Визначення 1

In [ ]:
import numpy
import matplotlib.pyplot as plt
import math
# from scipy.signal import 